![sutd](imgs/sutd.png)
## <center>50.040 Natural Language Processing, Summer 2019<center>
<center>**Homework 2**

<center>**Due 5 July 2019, 5pm** <center>

**Write your student ID and name**

ID: 1002323

Name: Woong Wen Tat

Students with whom you have discussed (if any):

### Requirements:
- Use Python to complete this homework.
- Please list students with whom you have discussed (if any).
- Follow the honor code strictly.
- Submit this ipynb file on eDimension before the deadline.

## Introduction
Constituency parsing aims to extract a constituency-based parse tree from a sentence that represents its syntactic structure according to a phrase structure grammar.

A typical constituency parse tree is shown below:

![tree](imgs/parse_tree.png)

$S$ is a distinguished start symbol, node labels such as $NP$(noun phrase), $VP$(verb phrase) are non-terminal symbols, leaf labels such as "a", "banana" are terminal symbols.

In this homework, we will be implementing a constituency parser based on probabilistic context-free grammars (PCFGs) and evaluate its performance.

## Dataset

We will be using a version of the [Penn Treebank](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.9.8216&rep=rep1&type=pdf) released in [NLTK corpora](http://www.nltk.org/nltk_data/) to induce PCFGs and evaluate our algorithm. 

The preprocessing code has been provided, **do not make any changes to the texts and code unless you are requested to do so.** Run the code below to load the training and test sets as Python lists, it will take ~1 minute.

Since we will not be tuning hyper-parameters for this homework, there will be no need for a development set.

In [1]:
import nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\Lumo\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [2]:
from util import get_train_test_data

In [3]:
#cnf_trees_train: training set, a list of parse trees
#cnf_trees_test: test set, a list of parse trees
cnf_trees_train, cnf_trees_test = get_train_test_data()

Each parse tree is of the [nltk.tree.Tree](https://www.nltk.org/_modules/nltk/tree.html) type and has been converted to the Chomsky normal form. Let us look at a sample parse tree in the training set:

In [4]:
print(cnf_trees_train[0])

(S
  (NP
    (NP (NNP pierre) (NNP vinken))
    (NP (, ,) (NP (ADJP (NP (CD 61) (NNS years)) (JJ old)) (, ,))))
  (S
    (VP
      (MD will)
      (VP
        (VB join)
        (VP
          (NP (DT the) (NN board))
          (VP
            (PP
              (IN as)
              (NP (DT a) (NP (JJ nonexecutive) (NN director))))
            (NP (NNP nov.) (CD 29))))))
    (. .)))


## PCFG

A probabilistic context-free grammar consists of:
- A context-free grammar $G=(N, \ \Sigma, \ S, \ R)$ where $N$ is a finite set of non-terminal symbols, $\Sigma$ is a finite set of terminal symbols, $R$ is a finite set of rules (e.g., $NP \rightarrow NP \ PP$), $S \in N$ is the start symbol.
- One parameter $q(A \rightarrow \beta)$ for each rule $A \rightarrow \beta$ in $R$. Since the grammar is in Chomsky normal form, there are only two types of rules: $A \rightarrow B \ C$ and $A \rightarrow \alpha$, where $A$, $B$, $C \in N$, $\alpha \in \Sigma$.

We can estimate the parameter $q(A \rightarrow \beta)$ using maximum likelihood estimation:

$$q_{MLE}(A \rightarrow \beta) = \frac {count(A \rightarrow \beta)}{count(A)}$$
where $count(A \rightarrow \beta)$ refers to the number of times we can observe the rule $A \rightarrow \beta$ in all the parse trees in the training set, and  $count(A)$  refers to the number of times we can see the non-terminal symbol $A$.

### Task 1 (6 points)

Starting from a single parse tree:
- First of all, let us only consider the first parse tree in the training set. List down all the unique grammar rules, unique non-terminal symbols, and unique terminal symbols that appear in this first parse tree from the training set.

Hint: [nltk.tree.Tree](https://www.nltk.org/_modules/nltk/tree.html) class provides many methods to get features of a tree. **Don't make any changes to the trees.**

In [5]:
#Check the methods of a parse tree of nltk.tree.Tree type
help(cnf_trees_train[0])

Help on Tree in module nltk.tree object:

class Tree(builtins.list)
 |  A Tree represents a hierarchical grouping of leaves and subtrees.
 |  For example, each constituent in a syntax tree is represented by a single Tree.
 |  
 |  A tree's children are encoded as a list of leaves and subtrees,
 |  where a leaf is a basic (non-tree) value; and a subtree is a
 |  nested Tree.
 |  
 |      >>> from nltk.tree import Tree
 |      >>> print(Tree(1, [2, Tree(3, [4]), 5]))
 |      (1 2 (3 4) 5)
 |      >>> vp = Tree('VP', [Tree('V', ['saw']),
 |      ...                  Tree('NP', ['him'])])
 |      >>> s = Tree('S', [Tree('NP', ['I']), vp])
 |      >>> print(s)
 |      (S (NP I) (VP (V saw) (NP him)))
 |      >>> print(s[1])
 |      (VP (V saw) (NP him))
 |      >>> print(s[1,1])
 |      (NP him)
 |      >>> t = Tree.fromstring("(S (NP I) (VP (V saw) (NP him)))")
 |      >>> s == t
 |      True
 |      >>> t[1][1].set_label('X')
 |      >>> t[1][1].label()
 |      'X'
 |      >>> print(t)
 |

In [6]:
#Write your code here
print ("rules", set(cnf_trees_train[0].productions()), '\n')
non_terminal_set = []
for s in cnf_trees_train[0].subtrees(lambda t: t.height() >= 2):
    non_terminal_set.append(s.label())
print ("unique non-terminals", set(non_terminal_set), '\n')
print ("unique terminals", set(cnf_trees_train[0].leaves()), '\n')

rules {NN -> 'board', VP -> PP NP, CD -> '29', S -> NP S, CD -> '61', NNS -> 'years', DT -> 'a', NP -> NNP NNP, PP -> IN NP, NP -> DT NN, IN -> 'as', NP -> ADJP ,, NP -> NNP CD, NNP -> 'nov.', NNP -> 'pierre', VP -> VB VP, NP -> DT NP, VP -> MD VP, MD -> 'will', JJ -> 'nonexecutive', NN -> 'director', , -> ',', ADJP -> NP JJ, NP -> NP NP, NNP -> 'vinken', JJ -> 'old', NP -> CD NNS, VB -> 'join', S -> VP ., NP -> JJ NN, . -> '.', DT -> 'the', NP -> , NP, VP -> NP VP} 

unique non-terminals {'PP', 'MD', 'NNP', ',', 'IN', 'DT', 'NNS', 'NN', 'JJ', 'VB', 'VP', 'NP', '.', 'S', 'CD', 'ADJP'} 

unique terminals {'join', ',', 'years', 'pierre', 'nonexecutive', 'old', 'as', 'nov.', '.', 'a', 'board', 'director', 'the', '61', 'vinken', 'will', '29'} 



There is an underlying PCFG used for parsing the sentences in the training set. To find out this PCFG, what we can do is to visit each parse tree that appears in the training set, and collect some useful information from each parse tree. Let us start with the following:
- Obtain all the grammar rules, non-terminal symbols, and terminal symbols.
- Show the numbers of unique non-terminal symbols, unique terminal symbols, and unique grammar rules. 
- List 10 most frequent grammar rules.

In [7]:
#Write your code here
from collections import Counter

terminal_counts = Counter()
non_terminal_counts = Counter()
grammar_rule_counts = Counter()

for tree in cnf_trees_train:
    for rule in tree.productions():
        grammar_rule_counts[rule] += 1
    for subtree in tree.subtrees(lambda t: t.height() >= 2):
        non_terminal_counts[subtree.label()] += 1
    for leaf in tree.leaves():
        terminal_counts[leaf] += 1

terminals = list(terminal_counts.keys())
non_terminals = list(non_terminal_counts.keys())

In [8]:
print (len(terminals), len(non_terminals), len(grammar_rule_counts.keys()))
print (grammar_rule_counts.most_common(10))

11226 74 14603
[(PP -> IN NP, 6117), (, -> ',', 4758), (DT -> 'the', 4590), (NP -> DT NP, 3914), (. -> '.', 3715), (S -> VP ., 2971), (NP -> NP NP, 2813), (NP -> DT NN, 2813), (NP -> NP PP, 2592), (S -> NP S, 2507)]


Store the unique terminal and non-terminal symbols in two separate lists "terminals" and "non-terminals" respectively. Store the grammar rules and their respective counts in a dictionary "grammar_rule_counts".

### Task 2 (8 points)

We can estimate the parameters (i.e., probabilities for the grammar rules) based on the counts collected from the training set. 
- Show the estimated parameter (i.e., probability) for each of the following rules: $S \rightarrow NP \ VP$, $NP \rightarrow DT \ NN$, $DT \rightarrow the$.

In [9]:
#Write your code here
#The original keys are in a NLTK Terminal/Non-terminal/Production class
non_terminal_counts_str = {str(k):v for k,v in non_terminal_counts.items()}
grammar_rule_counts_str = {str(k):v for k,v in grammar_rule_counts.items()}

for rules in ["S -> NP VP", "NP -> DT NN", "DT -> 'the'"]:
    print("estimated probability for ", rules, " = ", grammar_rule_counts_str[rules]/non_terminal_counts_str[rules.split(' ->')[0]])

estimated probability for  S -> NP VP  =  0.09527165932452276
estimated probability for  NP -> DT NN  =  0.07365995443714159
estimated probability for  DT -> 'the'  =  0.5819703309243058


It is possible that some sentences cannot be parsed under the PCFG learned from a limited training set. Take "Fruit flies like a banana" for example, assume its PCFG is:

$S → NP \ S \ (1)$

$NP → A \ N \ (0.5)$

$VP → V \ NP \ (1)$

$NP → D \ N \ (0.5)$

$A → Fruit \ (1)$

$N → flies \ (0.5)$

$V → like \ (1)$

$D→ a \ (1)$

$N → banana \ (0.5)$

In this case, we cannot construct a tree because there is no rule that can connect $S$ to $VP$.

Now, consider another PCFG as follows:

$S → NP \ VP \ (1)$

$NP → A \ N \ (0.5)$

$VP → V \ NP \ (1)$

$NP → D \ N \ (0.5)$

$A → Fruit \ (1)$

$N → flies \ (0.5)$

$V → like \ (1)$

$D→ the \ (1)$

$N → banana \ (0.5)$

In this case, we still cannot construct a tree because there is no rule for the terminal symbol "a". 

Let us do a simple check on one test sentence:
- Find out all terminal symbols of cnf_trees_test[0] that never appear in the PCFG rules that we have learned from the training set.

In [10]:
#Write your code here
for terminal in cnf_trees_test[0].leaves():
    if terminal_counts[terminal] == 0:
        print ("Unseen terminal symbol ",terminal)

Unseen terminal symbol  300-day


We can use smoothing techniques to handle these cases. A simple smoothing method is as follows. We first create a new "unknown" non-terminal symbol $UNK$ and a new "unknown" terminal symbol $unk$.

Next, for each original non-terminal symbol $A\in N$, we add two new rules $A \rightarrow UNK \ UNK$ and $A \rightarrow unk$ to the original PCFG.

For each terminal symbol $\alpha \in \Sigma$, we add one new rule $UNK \rightarrow \alpha$ to the original PCFG.

The smoothed probabilities for all rules can then be estimated as:
$$q_{smooth}(A \rightarrow \beta) = \frac {count(A \rightarrow \beta)}{count(A)+2}$$
$$q_{smooth}(A \rightarrow UNK \ UNK) = \frac {1}{count(A)+2}$$
$$q_{smooth}(A \rightarrow unk) = \frac {1}{count(A)+2}$$
$$q_{smooth}(UNK \rightarrow \alpha) = \frac {1}{|V|}$$
where $|V|$ is the count of unique terminal symbols, and the values of $count(\cdot)$ are the same as the ones used in Task 1 above.

- Add "$UNK$", "$unk$" to the two lists "non_terminals" and "terminals" respectively, compute and store the smoothed probabilities of grammar rules in a Python dictionary "smoothed_grammar_rule_probs". 
- Show the smoothed probability for each of the following rules: 

$\ \ \ \ \ \ \ \ \ \ S \rightarrow NP \ VP$

$\ \ \ \ \ \ \ \ \ \ NP \rightarrow DT  \ NN$

$\ \ \ \ \ \ \ \ \ \ DT \rightarrow the$

In [11]:
#Write your code to update the PCFG
from nltk.grammar import  Nonterminal
terminals.append('unk')
non_terminals.append(Nonterminal('UNK'))

In [12]:
from nltk.grammar import  Production

# +1 for each non_terminal and terminal first, we will iterate through this later
for non_terminal in non_terminals:
    if str(non_terminal) != 'UNK':
        grammar_rule_counts[Production(non_terminal, [Nonterminal('UNK'), Nonterminal('UNK')])] += 1
        grammar_rule_counts[Production(non_terminal, ['unk'])] += 1
for terminal in terminals:
    if terminal != 'unk':
        grammar_rule_counts[Production(Nonterminal('UNK'), [terminal])] += 1

In [13]:
print (len(grammar_rule_counts.keys()))
# difference of 1 is due to 'unk' and 'UNK' being added to the 2 list
print (len(grammar_rule_counts.keys()) - (len(terminals)-1) -  2*(len(non_terminals)-1))

25977
14603


In [14]:
#Write your code here
smoothed_grammar_rule_probs = {}
for rule, count in grammar_rule_counts.items():
    if str(rule.lhs()) == 'UNK':
        smoothed_grammar_rule_probs[rule] = count/len(terminals)
    else:
        smoothed_grammar_rule_probs[rule] = count/(non_terminal_counts[str(rule.lhs())]+2)

In [15]:
smoothed_grammar_rule_probs_str = {str(k):v for k,v in smoothed_grammar_rule_probs.items()}

for rules in ["S -> NP VP", "NP -> DT NN", "DT -> 'the'"]:
    print("estimated probability for ", rules, " = ", smoothed_grammar_rule_probs_str[rules])

estimated probability for  S -> NP VP  =  0.09526046866741059
estimated probability for  NP -> DT NN  =  0.07365609698620093
estimated probability for  DT -> 'the'  =  0.5818227912282926


## CKY Algorithm

### Task 3 (10 points)
Similar to the Viterbi algorithm, the CKY algorithm is a dynamic-programming algorithm. Given a PCFG $G=(N, \ \Sigma, \ S, \ R, \ q)$, we can use the CKY algorithm described in class to find the highest scoring parse tree for a sentence. 

First, let us complete the *CKY* function from scratch using only Python built-in functions and the Numpy package. 

The output should be two dictionaries $\pi$ and $bp$, which store the optimal probability and backpointer information respectively.

Given a sentence $w_0, w_1, ...,w_{n-1}$,  $\pi(i, k, X)$, $bp(i, k, X)$ refer to the highest score and backpointer for the (partial) parse tree that has the root X (a non-terminal symbol) and covers the word span $w_i, ..., w_{k-1}$, where $0 \le i < k \le n$. Note that a backpointer includes both the best grammar rule chosen and the best split point.
![tree](imgs/parse_tree.png)


Take "Fruit flies like a banana" for example, $\pi(0, 5, S)$ is the probability of the optimal complete parse tree, $bp(0, 5, S)$ is the corresponding backpointer. Specifically, the best split point is 2 and the best grammar rule chosen is $S \rightarrow NP \ VP$.

In [16]:
def CKY(sent, non_terminals, terminals, grammar_rule_probs):
    
    '''
    CKY algorithm
    args:
        sent: a sequence of words, list
        non_terminals: non-terminal symbols, list
        terminals: terminal symbols, list
        grammar_rule_probs: probabilities of the rules, dictionary
    returns:
        pi: highest score for (partial) parse tree, dictionary
        bp: backpointers, dictionary
    '''
    pi, bp ={}, {}
    n = len(sent)
    
    #from leaves to tree
    for level in range(1, n+1):
        #base case
        if level == 1: 

            # iterate through terminal tokens
            for pos, token in enumerate(sent):

                # best prob for each LHS at per-terminal level
                best_prob = Counter()
                
                # iterate through rules
                for rule, prob in grammar_rule_probs.items():
                    lhs, rhs = rule.lhs(), rule.rhs()
                    # for some ridiculous reason, some nonterminals are turned to string
                    if type(lhs) is str:
                        lhs = Nonterminal(lhs)
                    if rhs[0] == token:
                        pi[(pos, pos+1, lhs)] = prob
                        if prob >= best_prob[lhs]:
                            bp[(pos, pos+1, lhs)] = (rule, None)
        
        # recursive definition
        else:
            
            # iterate through (n - level + 1) partial trees
            for pos in range(n - level + 1):
                
                # best prob for each LHS at this level
                best_prob = Counter()
                    
                # iterate through (level - 1) split points
                for split in range(1, level):
                    
                    # iterate through rules
                    for rule, prob in grammar_rule_probs.items():
                        lhs, rhs = rule.lhs(), rule.rhs()
                        # for some ridiculous reason, some nonterminals are turned to string
                        if type(lhs) is str:
                            lhs = Nonterminal(lhs)
                        # only look at production rules that are not pre-terminal
                        if len(rhs) > 1:
                            if (pos, pos+split, rhs[0]) in pi and (pos+split, pos+level, rhs[1]) in pi:
                                prob = pi[(pos, pos+split, rhs[0])] * pi[(pos+split, pos+level, rhs[1])]
                                pi[(pos, pos+level, lhs)] = prob
                                if prob >= best_prob[lhs]:
                                    bp[(pos, pos+level, lhs)] = (rule, split)

    #Complete the code
    return pi, bp

Let us complete the *generate_parse_tree* function that can return a tree based on two arguments -- backpointers $bp$ and a "start node" $start$, where the "start node" is a tuple consisting of a span and a non-terminal symbol that covers that span. For example, if we set the start node as $(0, 5, S)$, the function will recursively build a tree based on the infomration stored in $bp$.

In [17]:
def generate_parse_tree(bp, start):
    '''
    Recursively construct a tree
    args:
        bp: backpointers that allow us to recover the highest scoring parse tree, dictionary
        start: the start node, i.e., (i, k, X), which refers to the non-terminal root X and the words span [i, k) it covers, tuple
    return:
        tree: the parse tree, nltk.tree.Tree type
    '''
    
    rule, split = bp[(start[0], start[1], start[2])]
    if split:
        left_child_start = (start[0], start[0] + split, rule.rhs()[0])
        right_child_start = (start[0] + split, start[1], rule.rhs()[1])
        return nltk.tree.Tree(start[2], [generate_parse_tree(bp, left_child_start), generate_parse_tree(bp, right_child_start)])
    else:
        return nltk.tree.Tree(start[2], rule.rhs())

Now using the PCFG learned from Task 2, we can work on the following:
- Compute the max probability for the sentence "the boy saw a nice cat" based on "terminals", "non_terminals", "smoothed_rule_probs" obtained in Task 2.
- Generate and display the parse tree of the sentence.

In [18]:
sent = "the boy saw a nice cat".split()
pi, bp = CKY(sent, non_terminals, terminals, smoothed_grammar_rule_probs)
tree =  generate_parse_tree(bp, (0, len(sent), Nonterminal('S')))
#Write your code to display max probability and generate the parse tree

In [19]:
print(tree)

(S
  (NP (ADJP (JJ the) (RB (UNK boy) (UNK saw))) (NN a))
  (UCP (UNK nice) (UNK cat)))


## Evaluate Our Parser

Now we can use the CKY algorithm implemented in Task 3 to predict parse trees on sentences from the test set, then evaluate the performance based on the gold parse trees of the test set. Before we perform the evaluation, we need to represent our parse trees in a form of constituents which are denoted as brackets $X(i, j)$, where $X$ is the non-terminal symbol **that has two children**, and $i$, $j$ refers to the starting point (inclusive) and ending point (exclusive) respectively. **(In other words, a constituent cannot consist of a single word only.)**

Metrics are defined as:
- Precision = (# of correct constituents in the predicted parse trees)/(# of total constituents in the predicted parse trees)

- Recall = (# of correct constituents in the predicted parse trees)/(# of correct constituents in the gold parse trees)

- F1 score = 2\*precision\*recall/(precision+recall)

Using the sentence "Fruit flies like a banana" as example again, there are five words in the sentence with labels from 0 to 4. Assume the constituents are $S(0, 5)$, $NP(0, 2)$, $VP(2, 5)$, $NP(3, 5)$ in the gold parse tree (shown in Task 3), and $S(0, 5)$, $NP(0, 2)$, $PP(2, 5)$, $NP(3, 5)$ in the predicted parse tree (shown below). Then the precision = 3/4, the recall = 3/4, and the F1 score = 0.75. 
![parse_tree](imgs/parse_tree2.png)

### Task 4 (6 points):
- Run the CKY algorithm on the test set and generate parse trees under the smoothed PCFG obtained in Task 2. Note that some test sentences contain words that are not included in the  list "terminals", set those words as $unk$.
- Extract the constituents from the gold and predicted parse trees respectively.
- Compute the overall precision, recall and F1 score for the test set.


In [20]:
#Write your code here
from collections import deque
gold_const = 0
pred_const = 0
corr_pred = 0
total = len(cnf_trees_test)

for idx, tree in enumerate(cnf_trees_test):
    print ("Evaluating %d/%d" %(idx +1 , total), end="\r")
    sent = [token if token in terminals else 'unk' for token in tree.leaves()]
    pi, bp = CKY(sent, non_terminals, terminals, smoothed_grammar_rule_probs)
    pred_tree =  generate_parse_tree(bp, (0, len(sent), Nonterminal('S')))
    gold_subtree_list = deque([(tree, 0, len(sent))])
    gold_const_list = []
    gold_const_list.append((Nonterminal('S'), 0, len(sent)))
    
    # queue system to go through subtrees
    while len(gold_subtree_list) > 0:
        gold_subtree, start, end = gold_subtree_list.popleft()
        left_child = gold_subtree[0]
        right_child = gold_subtree[1]
        split = start + len(left_child.leaves())
        
        if left_child.height() > 2:
            gold_const_list.append((left_child.label(), start, split))
            gold_subtree_list.append((left_child, start, split))
        if right_child.height() > 2:
            gold_const_list.append((right_child.label(), split, end))
            gold_subtree_list.append((right_child, split, end))
            
    pred_subtree_list = deque([(pred_tree, 0, len(sent))])
    pred_const_list = []
    pred_const_list.append((Nonterminal('S'), 0, len(sent)))
    
    # queue system to go through subtrees
    while len(pred_subtree_list) > 0:
        pred_subtree, start, end = pred_subtree_list.popleft()
        left_child = pred_subtree[0]
        right_child = pred_subtree[1]
        split = start + len(left_child.leaves())
        
        if left_child.height() > 2:
            pred_const_list.append((left_child.label(), start, split))
            pred_subtree_list.append((left_child, start, split))
        if right_child.height() > 2:
            pred_const_list.append((right_child.label(), split, end))
            pred_subtree_list.append((right_child, split, end))
    
    
    for const in pred_const_list:
        if const in gold_const_list:
            corr_pred += 1
    gold_const += len(gold_const_list)
    pred_const += len(pred_const_list)

In [21]:
recall = corr_pred/gold_const
recall

0.05150214592274678

In [22]:
precision = corr_pred/pred_const
precision

0.05150214592274678

In [23]:
f1_score = 2*precision*recall/(precision + recall)
f1_score

0.05150214592274678